In [2]:
import os
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
tqdm.pandas()

In [17]:
# 정책발행 파악
# 매월 발행(정책+민간) 진행
path = '../Data/Origin/20231208_사용자충전데이터.xlsx'

charge_df_origin1 = pd.read_excel(path, sheet_name = 'Result 1')
charge_df_origin1 = charge_df_origin1.rename(columns = {'month' : '기준연월'})
charge_df_origin2 = pd.read_excel(path, sheet_name = 'Result 2')
charge_df_origin2 = charge_df_origin2.rename(columns = {'month' : '기준연월'})

charge_df_origin = pd.concat([charge_df_origin1, charge_df_origin2], axis = 0)

In [6]:
charge_df_origin.head(2)

,회원구분,충전금액,발행(정책+민간),인센티브적립,인센티브사용,month,사용자코드
0,일반사용자,1000000,0,100000,0,201912,이윤주01058024525
1,가맹점주,53000,0,50,0,201912,이용인01023849506


In [4]:
charge_df_origin['구분'].unique()

array(['일반 사용자', '탈퇴한 사용자', '여성바우처', '기프트카드', '벼상토지원'], dtype=object)

In [5]:
charge_df = charge_df_origin.copy()
charge_df = charge_df.loc[(charge_df['구분'] == '여성바우처') & (charge_df['발행(정책+민간)'] != 0)]
charge_df = charge_df.groupby(['기준연월'])['발행(정책+민간)'].sum().reset_index()

charge_df.head(2)

,기준연월,발행(정책+민간)
0,202104,1.142140e+09
1,202105,2.928340e+06


In [6]:
charge_df = charge_df_origin.copy()
charge_df = charge_df.loc[(charge_df['구분'] == '기프트카드') & (charge_df['발행(정책+민간)'] != 0)]
charge_df = charge_df.groupby(['기준연월'])['발행(정책+민간)'].sum().reset_index()

charge_df.head(2)

,기준연월,발행(정책+민간)


In [7]:
charge_df = charge_df_origin.copy()
charge_df = charge_df.loc[(charge_df['구분'] == '벼상토지원') & (charge_df['발행(정책+민간)'] != 0)]
charge_df = charge_df.groupby(['기준연월'])['발행(정책+민간)'].sum().reset_index()

charge_df.head(2)

,기준연월,발행(정책+민간)
0,202302,1.538262e+09
1,202303,7.989000e+06


In [8]:
charge_df = charge_df_origin.copy()
charge_df = charge_df.loc[(charge_df['구분'] == '일반 사용자') & (charge_df['발행(정책+민간)'] != 0)]
charge_df = charge_df.groupby(['기준연월'])['발행(정책+민간)'].sum().reset_index()

charge_df.head(2)

,기준연월,발행(정책+민간)
0,202011,3.429700e+07
1,202012,4.774614e+09


In [27]:
charge_df_origin = charge_df_origin.groupby(['기준연월'])['발행(정책+민간)'].sum().reset_index()
charge_df_origin = charge_df_origin.rename(columns = {'발행(정책+민간)' : '가입월 발행'})
charge_df_origin['가입 -1 월 발행'] = charge_df_origin['가입월 발행'].shift(1)
charge_df_origin['가입 -2 월 발행'] = charge_df_origin['가입월 발행'].shift(2)
charge_df_origin['가입 -3 월 발행'] = charge_df_origin['가입월 발행'].shift(3)
charge_df_origin['가입 -4 월 발행'] = charge_df_origin['가입월 발행'].shift(4)

In [10]:
# 가맹후 6개월 간의 매출 증감율 파악

In [32]:
path = '../Data/Preprocessed/순환분석.csv'

df_franchise_origin = pd.read_csv(path)

basic_columns = ['가맹점 사업자 번호', '가맹점 업종', '가맹점 가맹점명', '가맹점 골목상권분류', '가맹점 대분류', '가맹점 중분류', '가맹점 소분류', '가맹점 가맹점 주소', '가맹점 가맹가입일자']
tmp01 = df_franchise_origin[basic_columns]
tmp02 = df_franchise_origin[['사용자 사업자 번호', '사용자 업종', '사용자 가맹점명', '사용자 골목상권분류', '사용자 대분류', '사용자 중분류', '사용자 소분류', '사용자 가맹점 주소', '사용자 가맹가입일자']]

for column01, column02 in zip(sorted(tmp01.columns.tolist()), sorted(tmp02.columns.tolist())):
    tmp02 = tmp02.rename(columns = {column02 : column01})

tmp01 = tmp01.drop_duplicates()
tmp02 = tmp02.drop_duplicates()
df_franchise = pd.concat([tmp01, tmp02], axis = 0, ignore_index = True)
df_franchise = df_franchise.loc[(df_franchise['가맹점 사업자 번호'] != '삭제된가맹점')]

In [33]:
df_franchise.columns

Index(['가맹점 사업자 번호', '가맹점 업종', '가맹점 가맹점명', '가맹점 골목상권분류', '가맹점 대분류', '가맹점 중분류',
       '가맹점 소분류', '가맹점 가맹점 주소', '가맹점 가맹가입일자'],
      dtype='object')

In [34]:
path = '../Data/Preprocessed/사용자분석.csv'

df_sales_origin = pd.read_csv(path)

df_sales_origin = df_sales_origin.drop(columns = ['월', '요일'])
df_sales_origin['날짜'] = pd.to_datetime(df_sales_origin['날짜'], format = '%Y%m%d')
df_sales_origin = df_sales_origin.loc[df_sales_origin['가맹점 주소'].str[-1].isin(['면', '읍'])]
df_sales_origin = df_sales_origin.merge(df_franchise, on = '가맹점 사업자 번호', how = 'left')
df_sales_origin['가맹점 읍면 구분'] = df_sales_origin['가맹점 주소'].progress_apply(lambda x : x if x in ['부여읍', '홍산면', '규암면'] else '기타')
# df_sales_origin = df_sales_origin.loc[(df_sales_origin['가맹점 대분류'] != '기타')]

100%|██████████| 10569218/10569218 [00:10<00:00, 1047176.60it/s]


In [43]:
basic_columns = ['가맹점 사업자 번호', '가맹점명', '가맹점 읍면 구분', '가맹점 주소', '골목상권분류', '대분류', '중분류', '소분류']

basic_df = df_sales_origin[basic_columns + ['가맹점 가맹가입일자']].drop_duplicates()
basic_df = basic_df.loc[(basic_df.isnull().any(axis = 1) != True)]

In [44]:
basic_df['가맹점 가맹가입일자'] = pd.to_datetime(basic_df['가맹점 가맹가입일자'], format = '%Y-%m-%d')


def month_plus(date):
    if date.day > 28:
        date = date.replace(day = 28)

    try:
        date = date.replace(month = date.month + 1)
    except:
        date = date.replace(year = date.year + 1, month = 1)
    date = date.replace(day = 1)

    return date

basic_df['가맹 후 첫 달의 날짜'] = basic_df['가맹점 가맹가입일자'].apply(lambda x : month_plus(x))
basic_df['첫 달의 첫 날짜 사이의 시간'] = basic_df['가맹 후 첫 달의 날짜'] - basic_df['가맹점 가맹가입일자']
basic_df['가맹 첫 달의 영업정도'] = basic_df.apply(lambda x : round((x['첫 달의 첫 날짜 사이의 시간']) /
                                                                  (x['가맹 후 첫 달의 날짜'] - x['가맹점 가맹가입일자'].replace(day = 1)) * 100, 2), axis = 1)
basic_df['첫 달 추가집계 여부'] = basic_df['가맹 첫 달의 영업정도'] >= 50


In [47]:
basic_df.columns

Index(['가맹점 사업자 번호', '가맹점명', '가맹점 읍면 구분', '가맹점 주소', '골목상권분류', '대분류', '중분류',
       '소분류', '가맹점 가맹가입일자', '가맹 후 첫 달의 날짜', '첫 달의 첫 날짜 사이의 시간', '가맹 첫 달의 영업정도',
       '첫 달 추가집계 여부'],
      dtype='object')

In [48]:
basic_df['첫 달 집계'] = np.nan
for i in range(1, 7):
    basic_df[f'임시__{i} 달의 매출'] = np.nan

count_zero_list = []
for k, (i, row) in enumerate(tqdm(basic_df.iterrows())):
    tmp_df = df_sales_origin.loc[(df_sales_origin['가맹점 사업자 번호'] == row['가맹점 사업자 번호']), :]
    first_df = tmp_df.loc[(tmp_df['가맹점 가맹가입일자'] <= tmp_df['날짜']) & (tmp_df['날짜'] < row['가맹 후 첫 달의 날짜']), :]
    remain_df = tmp_df.loc[(row['가맹 후 첫 달의 날짜'] <= tmp_df['날짜']), :]    
    
    first_real_sum = first_df['사용금액'].sum()
    remain_df = remain_df.set_index('날짜')
    
    first_date_year = row['가맹점 가맹가입일자'].year
    first_date_month = row['가맹점 가맹가입일자'].month

    if first_date_month + 7 > 12:
        tmp_year = first_date_year + 1
        tmp_month = (first_date_month + 7) % 12

    else:
        tmp_year = first_date_year
        tmp_month = first_date_month + 7
    

    tmp_date = pd.to_datetime(str(tmp_year).zfill(4) + str(tmp_month).zfill(2) + '01', format = '%Y%m%d')  
    remain_df = remain_df.resample('1m')['사용금액'].sum()
    remain_df = remain_df.loc[:tmp_date]

    count_value = len(remain_df)
    if len(remain_df) == 0:
        basic_df.loc[i, basic_df.columns.tolist()[-6 : ]] = [np.nan for _ in range(6)]
    else:
        basic_df.loc[i, basic_df.columns.tolist()[-6 : ]] = remain_df.values.tolist() + [np.nan] * (6 - count_value)

    basic_df.loc[i, '첫 달 집계'] = first_real_sum

# basic_df.to_csv('../Data/Preprocessed/가맹효과.csv', encoding = 'utf-8-sig', index = False)

2418it [25:21,  1.59it/s]


In [64]:
basic_df = pd.read_csv('../Data/Preprocessed/가맹효과.csv', encoding = 'utf-8-sig')

basic_df.loc[:, basic_df.columns[-7:]] = basic_df.loc[:, basic_df.columns[-7:]].fillna(0).values
basic_df['수정__첫 달 집계'] = basic_df.progress_apply(lambda x : round(x['첫 달 집계'] * 100 / x['가맹 첫 달의 영업정도']), axis = 1)
basic_df = basic_df[basic_df.columns[:-7].tolist() + [basic_df.columns[-1]] + basic_df.columns[-7:-1].tolist()]

100%|██████████| 2418/2418 [00:00<00:00, 79546.86it/s]


In [66]:
basic_df.columns

Index(['가맹점 사업자 번호', '가맹점명', '가맹점 읍면 구분', '가맹점 주소', '골목상권분류', '대분류', '중분류',
       '소분류', '가맹점 가맹가입일자', '가맹 후 첫 달의 날짜', '첫 달의 첫 날짜 사이의 시간', '가맹 첫 달의 영업정도',
       '첫 달 추가집계 여부', '첫 달 집계', '수정__첫 달 집계', '임시__1 달의 매출', '임시__2 달의 매출',
       '임시__3 달의 매출', '임시__4 달의 매출', '임시__5 달의 매출', '임시__6 달의 매출'],
      dtype='object')

In [67]:
sum_columns = basic_df.columns[-7:]
last_columns = basic_df.columns[:9]

save_path = '../Data/Preprocessed/가맹효과_데이터.csv'
if os.path.isfile(save_path):
    os.remove(save_path)

pd.DataFrame(columns = last_columns.to_list() + sum_columns.to_list() + ['가입월 발행', '가입 -1 월 발행', '가입 -2 월 발행', '가입 -3 월 발행', '가입 -4 월 발행']).to_csv(encoding = 'utf-8-sig', escapechar = '\\')

for i, row in tqdm(basic_df.iterrows()):
    tmp = row[sum_columns]
    tmp.name = '매출액'
    tmp = tmp.to_frame()
    tmp['전달_매출'] = tmp['매출액'].shift(1)
    tmp['데이터 적합성 여부'] = (tmp[['매출액', '전달_매출']] != 0).all(axis = 1)
    tmp['매출액 변화'] = tmp['매출액'] - tmp['전달_매출']
    tmp['매출액 변화율'] = tmp.apply(lambda x : np.nan if (x['데이터 적합성 여부'] == False) |
                                                         (pd.isnull(x['전달_매출']) == True) else round(x['매출액 변화'] / x['전달_매출'] * 100, 2), axis = 1)

    tmp =  tmp.iloc[1:]
    tmp['구분'] = [f'{i + 1}달 ~ {i + 2}달' for i in range(0, 6)]

    standard_date = row['가맹점 가맹가입일자'].replace('-', '')
    standard_year_v = int(standard_date[:4])
    standard_month_v = int(standard_date[4:6])

    for l, (k, row_k) in enumerate(tmp.iterrows()):
        standard_month = standard_month_v + l
        
        if standard_month > 12:
            standard_month = standard_month % 12
            standard_year = standard_year_v + 1
        else:
            standard_year = standard_year_v

        publish = charge_df_origin.loc[(charge_df_origin['기준연월'] == int(str(standard_year) + str(standard_month).zfill(2))), charge_df_origin.columns[1:]]
        publish.index = range(len(publish))

        concat_row = pd.concat([row[last_columns], row_k]).to_frame().T
        concat_row.index = range(len(concat_row))
        concat_row = pd.concat([concat_row, publish], axis = 1)

        concat_row.to_csv(save_path, index = False, encoding = 'utf-8-sig', escapechar = '\\', mode = 'a', header = None)

0it [00:00, ?it/s]

2418it [00:48, 49.68it/s]
